# Walnut
### Des prédicats aux automates, visite dans les entrailles de walnut.

#### <b class="cool">Samuel Nalin</b>
<img style="width:50%; height: auto;" src="img/lifo_and_univ.png">

### 27 mars 2023, Squirrel club 🐿
$\renewcommand{\S}[1]{\Sigma_{{#1}}}$ 
$\renewcommand{\Sp}[1]{\Sigma_{{#1}}'}$ 
$\renewcommand{\Spp}[1]{\Sigma_{{#1}}''}$

## En bref

Walnut est un outil qui permet d'évaluer des formules de l'arithmétique de Büchi, pour ce faire la formule considérée est lue en forme postfixe et chaque sous formule est traitée comme une opération sur des automates finis déterministes.

Cette présentation a pour but de présenter cette évaluation, de décrire  en pratique comment ces opérations sont traitées par Walnut.
(*i. e.* l'effet des commandes <b class="cool">def</b> et <b class ="cool">eval</b>).

## Système numération

Walnut permet l'utilisation de différents <b class="cool">systèmes de numération</b>.

Pour définir un système de numération à Walnut, il est nécessaire de décrire les automates  de la <b class="cool"> représentation </b> et de <b class="cool">l'addition</b>. 
Walnut considère ensuite l'ordre lexicographique pour traiter <b class="cool">l'égalité</b> et la <b class="cool">comparaison</b>.

Remarque : dans tout système de numération, les nombres $0$ et $1$ doivent être codés par $0$ et $1$ respectivement.

Par exemple, pour msd_2, les automates sont les suivants :

<img style="height:100%;" src="img/msd_2/Rmsd2.svg">
<img style="height:100%; position:absolute; float:left; left:0px; bottom:0px;" src="img/msd_2/Rmsd2.svg">
<img style="height:100%; position:absolute; float:right; right:0px; bottom:0px" src="img/msd_2/equalmsd2.svg">

<img style="height:100%;" src="img/msd_2/plusmsd2.svg">
<img style="height:100%; position:absolute; float:left; left:0px; bottom:0px;" src="img/msd_2/plusmsd2.svg">
<img style="height:100%; position:absolute; float:right; right:0px; bottom:0px" src="img/msd_2/leqmsd2.svg">

## Formules, DFA, même combat

Walnut traduit les <b class="cool">prédicats</b> qui lui sont donnés en des <b class="cool">automates finis déterministes</b>.

Un <b class="gelb">terme arithmétique</b> peut être :
* une constante arithmétique ($\in \mathbb{N}$),
* une variable,
* $x_1+x_2$ ou $x_1-x_2$ avec $x_1$ et $x_2$ des termes arithmétiques,
* $x*n$ ou $n*x$ ou $x/n$ avec $x$ un terme arithmétique et $n$ une constante arithmétique.

Un <b class="blau">terme d'indexation</b> peut être :
* une constante alphabétique,
* $W[e_1]\cdots[e_m]$ l'indexation d'un mot automatique $W$ avec $e_i$ un terme arithmétique ou un prédicat à une variable libre.

Un <b class="cool">prédicat</b> peut être :
* $t_1 \triangleleft t_2$ avec $t_1$ et $t_2$ des termes de même type et $\triangleleft \in \lbrace =,<,<=,>,>=\rbrace$ un opérateur de comparaison,
* $\$P(e_1,\cdots,e_m)$ l'appel à un prédicat $P$ avec $e_i$ un terme arithmétique ou un prédicat à une variable libre,
* $\sim P$ avec $P$ un prédicat,
* $P_1 \diamond P_2$ avec $P_1$ et $P_2$ des prédicats et $\diamond \in \lbrace \&, |, \hat~, =>, <=> \rbrace$ un opérateur logique,
* $\text{E}x P$ ou $\text{A}x P$ avec $P$ un prédicat et $x$ une variable libre.

Un <span class="cool">prédicat</span> $P$ de variables libres $x_1,...,x_n$ est représenté par le <b class="cool">DFA</b> $M = (Q, \S{1},\cdots,\S{n}, q_0, F, \delta)$ avec
* $Q$ son ensemble fini d'états,
* $\S{i}$ l'alphabet de $S_i$ avec $1 \leq i \leq n$ et où $S_i$ est le système de numération de $x_i$,
* $q_0 \in Q$ son état initial,
* $F$ ses états acceptants,
* $\delta : Q \times \S{1} \times \cdots \times \S{n} \rightarrow Q$ sa fonction partielle de transition.

In [42]:
def factorEq "Ak k<n => T[i+k]=T[j+k]";
eval thueinf "At,i Ej (j>i) & $factorEq(i,j,t)";


____
TRUE



Illustrons sur l'exemple suivant le cheminement de Walmut pour évaluer une formule.

In [1]:
def factorEq "Ak k<n => T[i+k]=T[j+k]";

La formule est transformée sous forme postfixe dans une file :

<img style="width:100%" src="img/lifo/FIFO.svg">

La formule est reconstituée sous forme d'automate au moyen d'une pile, dont voici l'évolution sur l'exemple :

<img style="width:100%" src="img/lifo/LIFO.svg">

Après chaque étape, le DFA obtenu est minimisé et émondé de son état puit.

## Plan de route

Il est donc nécessaire de décrire toutes les opérations sur les automates utilisées par Walnut :

* <span class="">Les opérations sur les prédicats,</span>
    * <b class="cool">Les opérateurs de logique d'arité 2</b>
    * <span class="">la négation</span>
    * <span class="">La quantification</span>
* <span class="">Les opérations sur les termes arithmétiques,</span>
* <span class="">L'appel à des prédicats préfinis,</span>
* <span class="">Les opérations sur les termes d'indexation de mots automatiques.</span>

## Produit synchronisé : outil clef des opérateurs logiques

Soient $M = (Q, \S{1},\cdots,\S{m}, q_0, F, \delta)$ et $M' = (Q', \Sp{1},\cdots,\Sp{n}, q_0', F', \delta')$ les automates représentant les prédicats $P(x_1, \cdots, x_m)$ et $P'(x_1', \cdots, x_n')$ 
avec $S_i=S'_j$ si $x_i=x_j'$.

On cherche à construire l'automate $M''$ représentant le prédicat $(P\diamond P')(x_1'', \cdots, x_p'')$ avec $\diamond \in \lbrace \land, \lor, \oplus, \Rightarrow, \Leftrightarrow \rbrace$ et $\lbrace x_1'' \cdots x_p'' \rbrace = \lbrace x_1 \cdots x_m \rbrace \cup \lbrace x_1' \cdots x_n' \rbrace$ 
(ordonné lexicographiquement).

On construit le DFA $A = (Q'', \Spp{1}, \cdots, \Spp{p}, q_0'', F'', \delta'')$ avec :

* $Q'' = Q \times Q'$,
* $\Spp{k} = \S{i}$ si $x_k''=x_i$ ou  $\Spp{k} =\Sp{j}$ si $x_k''=x_j'$,
* $q_0'' = (q_0,q_0')$,
* $F'' = \lbrace (q,q') ~|~ (q \in F) \diamond (q' \in F')\rbrace$,
* $\delta''((q,q'), c_1, \cdots, c_p) = (\delta(q, a_1, \cdots, a_m), \delta'(q', b_1, \cdots, b_n))$
avec $a_i \in \S{i}, b_j \in \Sp{i}, c_k \in \Spp{i}$, $a_i=c_k$ si $x_i=x_k''$ et $b_j=c_k$ si $x_j'=x_k''$.

<b class="cool">$M''$ est l'automate résultant de la minimisation de $A$.</b>

#### Exemple pour  $~(a,b): a=1~ \& ~b=2$

<img style="height:100%;" src="img/synchro/a=1.svg">
<img style="height:100%; position:absolute; float:left; left:0px; bottom:0px;" src="img/synchro/a=1.svg">
<img style="height:100%; position:absolute; float:right; right:0px; bottom:0px" src="img/synchro/b=2.svg">

produit synchronisé avec $F'' = \lbrace (q,q') ~|~ (q \in F) \land (q' \in F')\rbrace$ :

<img style="height:100%" src="img/synchro/a=1crossb=2.svg">

Minimisation :

<img style="height:100%" src="img/synchro/a=1andb=2.svg">

C'est effectivement ce que fait Walnut :

In [44]:
eval testET "a=1 & b=2"::

computing a=1
computed a=1
a=1:2 states - 0ms
 computing b=2
 computed b=2
 b=2:3 states - 1ms
  computing a=1&b=2
   computing &:2 states - 3 states
   computing cross product:2 states - 3 states
   computed cross product:6 states - 0ms
-----     Minimizing: 6 states.
     determinizing:6 states
     determinized:6 states - 1ms
-----     Minimized:3 states - 1ms.
    Minimized:3 states - 1ms.
   computed &:3 states - 4ms
  computed a=1&b=2
  (a=1&b=2):3 states - 4ms
Total computation time: 5ms.



In [45]:
%showme testET

⚠️ Pour toute opération autre que la conjonction (le &), il est nécessaire que les automates soient <b class="cool">complets</b> !

Walnut purge ses automates de leurs états puit. Une étape de complétion est donc effectuée sur chaque DFA avant le produit synchronisé.

In [46]:
eval testOU "a=1 | b=2"::

computing a=1
computed a=1
a=1:2 states - 0ms
 computing b=2
 computed b=2
 b=2:3 states - 0ms
  computing a=1|b=2
   computing |:2 states - 3 states
    totalizing:2 states
    totalized:3 states - 0ms
    totalizing:3 states
    totalized:4 states - 0ms
   computing cross product:3 states - 4 states
   computed cross product:12 states - 0ms
-----     Minimizing: 12 states.
     determinizing:12 states
     determinized:12 states - 1ms
-----     Minimized:9 states - 1ms.
    Minimized:9 states - 1ms.
   computed |:9 states - 1ms
  computed a=1|b=2
  (a=1|b=2):9 states - 2ms
Total computation time: 2ms.



In [47]:
%showme testOU

## Plan de route

* <span class="">Les opérations sur les prédicats,</span>
    * <span class="gray">Les opérateurs de logique d'arité 2</span>
    * <b class="cool">la négation</b>
    * <span class="">La quantification</span>
* <span class="">Les opérations sur les termes arithmétiques,</span>
* <span class="">L'appel à des prédicats préfinis,</span>
* <span class="">Les opérations sur les termes d'indexation de mots automatiques.</span>

## Négation logique : complémentaire et produit synchronisé

La négation d'un prédicat est obtenue en prenant le <b class="cool">complémentaire</b> de son DFA tout en s'assurant que ne sont acceptées que les entrées <b class="cool">valides</b> dans le système de numération considéré.

Soit $M = (Q, \S{1},\cdots,\S{m}, q_0, F, \delta)$ un automate représentant le prédicat $P(x_1, \cdots, x_m)$.
Comme énoncé précédement, Walnut élague ses automates de leur état puit, $\delta$ est donc en réalité une fonction partielle.
Construisons l'automate $M'= (Q', \S{1},\cdots,\S{m}, q_0, F', \delta')$ reconnaissant le complémentaire de $P(x_1, \cdots, x_m)$.


* $Q' = Q \cup \lbrace \bot \rbrace$,
* $F' = Q' \setminus F$
* $\delta'(q, a_1, \cdots, a_m) = \delta(q, a_1, \cdots, a_m)$ si cette transition est définie et $\bot$ sinon,
avec $a_i \in \S{i}$ et $q \in Q'$.


#### Exemple pour $(a):~ ?\text{msd_fib} \sim a=1$

<img style="height:100%" src="img/complement/a=1.svg">

On complète l'automate :

<img style="height:100%" src="img/complement/a=1TOT.svg">

L'automate $M'$ complémentaire de $M$ :

<img style="height:100%" src="img/complement/a=1SWITCH.svg">


Ce DFA accepte $11$, cela pose problème en base de fibonacci.

Il faut donc intersecter $M'$ avec l'automate reconnaissant les nombres représentés dans la base de fibonacci.

<img style="height:100%" src="img/complement/a=1SWITCH.svg">
<img style="height:100%; position:absolute; float:left; left:0px; bottom:0px;" src="img/complement/a=1SWITCH.svg">
<img style="height:100%; position:absolute; float:right; right:0px; bottom:0px" src="img/complement/Rfib.svg">

La conjonction de ces deux automates donne le DFA pour le prédicat énoncé :
il reconnaît tous les nombres en base de fibonacci sauf le $1$.

<img style="height:100%" src="img/complement/a=1NEG.svg">

## Automates vrai - faux : le cas particulier

Les prédicats s'évaluant à <b class="cool"> vrai </b> ou à <b class="cool"> faux </b> sont représentés par des automates spéciaux à zéro variables.

In [2]:
eval true "Ax x=x";
eval false "Ex x<0";

____
TRUE

computed ~:1 states - 23ms
_____
FALSE



In [3]:
%showme true
%showme false

Dans la théorie ces automates ne sont pas des cas particuliers, mais Walnut les traite comme tels : toutes les opérations logiques impliquant un automate vrai/faux sont traitées de manière immédiate.

In [50]:
eval test "(Ax x=x) => y=2"::

computing x=x
computed x=x
x=x:1 states - 0ms
 computing quantifier A
  computing ~:1 states
   totalizing:1 states
   totalized:1 states - 0ms
-----    Minimizing: 1 states.
    determinizing:1 states
    determinized:1 states - 0ms
-----    Minimized:1 states - 0ms.
   Minimized:1 states - 0ms.
  computed ~:1 states - 1ms
  quantifying:1 states
 computed quantifier (A x x=x)
 (A x x=x):1 states - 1ms
  computing y=2
  computed y=2
  y=2:3 states - 0ms
   computing (A x x=x)=>y=2
   computed (A x x=x)=>y=2
   ((A x x=x)=>y=2):3 states - 0ms
Total computation time: 2ms.



## Plan de route

* <span class="">Les opérations sur les prédicats,</span>
    * <span class="gray">Les opérateurs de logique d'arité 2</span>
    * <span class="gray">la négation</span>
    * <b class="cool">La quantification</b>
* <span class="">Les opérations sur les termes arithmétiques,</span>
* <span class="">L'appel à des prédicats préfinis,</span>
* <span class="">Les opérations sur les termes d'indexation de mots automatiques.</span>

## Quantification : non-déterminisme et 0 piégeux

Soit $M= (Q, \S{1},\cdots,\S{m}, q_0, F, \delta)$ le DFA représentant le prédicat $P(x_1, \cdots, x_m)$.
Pour un $1 \leq i \leq m$, ajouter un <b class="cool">quantificateur existentiel</b> sur la variable libre $x_i$ revient à <b class="cool">projeter</b> dans $(\S{1}, \cdots, \S{m})$.
L'automate $M'= (Q, \S{1},\cdots,\S{m}, q_0, F, \delta')$ ainsi obtenu est fini potentiellement non déterministe avec 

* $\delta'(q, a_1, \cdots, a_{i-1}, a_{i+1}, \cdots, a_m) = 
\lbrace \delta(q, a_1, \cdots, a_{i-1}, a_i, a_{i+1}, \cdots, a_m) ~|~ a_i \in \S{i}\rbrace$.

#### Exemple avec $(a): \text{E}b ~ a=1 \& b=2$

<img style="height:100%" src="img/quantification/a=1andb=2.svg">

Projection :

<img style="height:100%" src="img/quantification/E(M,2).svg">

Ce NFA ne correspond pas à $(a): \text{E}b ~ a=1 \& b=2$.
Certaines représentations passent au travers des mailles du filet, en effet $01$ est accepté alors que $1$ non alors qu'ils sont tous deux des représentations valides d'un même nombre en msd_2.

Il est nécessaire de gérer ces <b class="cool">$0$ non significatifs</b>. En msd ils sont prefixes de la représentation et en lsd ils sont suffixes, deux techniques différentes s'imposent :

* msd : transformer en état entrant tous les états accessibles en lisant $(0,\cdots,0)^*$,
* lsd : transformer en état acceptant tous les états pouvant accéder à un état acceptant en lisant $(0,\cdots,0)^*$.

Poursuivant notre exemple, cela donne :

<img style="height:100%" src="img/quantification/Eba=1andb=2.svg">

Une fois déterminisé, on obtient bien $(a): \text{E}b ~ a=1 \& b=2$

<img style="height:100%" src="img/quantification/Eba=1andb=2DET.svg">

Ainsi l'automate M' représentant le le prédicat $(\text{E}x_i P)(x_1, \cdots,x_{i-1},x_{i+1}, \cdots, x_m)$ est la minimisation de l'automate $A= (Q, \S{1},\cdots,\S{m}, I, F', \delta')$ avec

* $\delta'(q, a_1, \cdots, a_{i-1}, a_{i+1}, \cdots, a_m) = 
\lbrace \delta(q, a_1, \cdots, a_{i-1}, a_i, a_{i+1}, \cdots, a_m) ~|~ a_i \in \S{i}\rbrace$,
* $I = \lbrace \delta'^*(q_0,0^k,\cdots,0^k) \neq \bot ~|~ k \geq 0 \rbrace$ en msd et $\lbrace q_0 \rbrace$ en lsd,
* $F = F$ en msd et $\lbrace q ~|~ \exists k,~ \delta^{\prime *}(q,0^k,\cdots,0^k) \in F \rbrace$ en lsd.


Remarque : Si plusieurs variables libres sont capturées par un même quantificateur universel Walnut projette sur toutes en même temps, nécessitant ainsi qu'une seule minimisation. 
(écrire $"\text{E}x,y,z~ \$\text{pred}(x,y,z)"$ est plus efficace que $"\text{E}x~ \text{E}y~ \text{E}z~ \$\text{pred}(x,y,z)"$ )

Pour le <b class="cool">quantificateur universel</b>, à partir de l'automate du prédicat $P(x_1, \cdots, x_m)$,
le prédicat $(\text{A}x_i~ P)(x_1, \cdots,x_{i-1},x_{i+1}, \cdots, x_m)$ est construit par négation, quantification existentielle et négation.

En effet, $(\text{A}x_i~ P)(x_1, \cdots,x_{i-1},x_{i+1}, \cdots, x_m)$ est équivalent à 
$(\sim (\text{E}x_i~ \sim P))(x_1, \cdots,x_{i-1},x_{i+1}, \cdots, x_m)$.

C'est bien ce que Walnut fait :

In [10]:
eval univ "Ab (a=1 & b=2)"::

computing a=1
computed a=1
a=1:2 states - 1ms
 computing b=2
 computed b=2
 b=2:3 states - 0ms
  computing a=1&b=2
   computing &:2 states - 3 states
   computing cross product:2 states - 3 states
   computed cross product:6 states - 0ms
-----     Minimizing: 6 states.
     determinizing:6 states
     determinized:6 states - 0ms
-----     Minimized:3 states - 0ms.
    Minimized:3 states - 0ms.
   computed &:3 states - 0ms
  computed a=1&b=2
  (a=1&b=2):3 states - 0ms
   computing quantifier A
    computing ~:3 states
     totalizing:3 states
     totalized:4 states - 0ms
-----      Minimizing: 4 states.
      determinizing:4 states
      determinized:4 states - 0ms
-----      Minimized:4 states - 1ms.
     Minimized:4 states - 1ms.
    computed ~:4 states - 1ms
    quantifying:4 states
-----      Minimizing: 4 states.
      determinizing:4 states
      determinized:5 states - 0ms
-----      Minimized:1 states - 0ms.
     Minimized:1 states - 0ms.
    quantified:1 states - 0ms
    fixin

## Plan de route

* <span class="gray">Les opérations sur les prédicats,</span>
    * <span class="gray">Les opérateurs de logique d'arité 2</span>
    * <span class="gray">la négation</span>
    * <span class="gray">La quantification</span>
* <b class="cool">Les opérations sur les termes arithmétiques,</b>
* <span class="">L'appel à des prédicats préfinis,</span>
* <span class="">Les opérations sur les termes d'indexation de mots automatiques.</span>

## Opérateurs de comparaison : le ruissellement

Maintenant que les opérateurs logiques sont définis il est aisé de traiter les comparaisons :

* $(a,b):~ ?S~ a<=b$ est équivalent à $(a,b):~ ?S~ a<b ~|~ a=b$,
* $(a,b):~ ?S~ a>b$ est équivalent à $(a,b):~ ?S~ \sim(a<=b)$,
* $(a,b):~ ?S~ a>=b$ est équivalent à $(a,b):~ ?S~ a>b ~|~ a=b$.

Comme Walnut transforme toutes les sous formules en automates, il est aussi possible de traiter des comparaisons entre predicats plus compliqués au moyen de quantificateur :

$(a,b):~ ?S~ 5 > a+b+4-1$ équivaut à $(a,b):~ ?S~ \text{E}c~ \$\text{aux}(a,b,c) ~\&~ 5>c$ avec $\text{aux}$ étant $(a,b,c):~ ?S~ c = a+b+4-1$.

In [53]:
eval comp "5>a+b+4-1"::

computing a+b
computed a+b
computing (a+b)+4
 computing &:4 states - 2 states
 computing cross product:4 states - 2 states
 computed cross product:8 states - 0ms
-----   Minimizing: 8 states.
   determinizing:8 states
   determinized:8 states - 0ms
-----   Minimized:7 states - 1ms.
  Minimized:7 states - 1ms.
 computed &:7 states - 1ms
 quantifying:7 states
-----   Minimizing: 7 states.
   determinizing:7 states
   determinized:5 states - 0ms
-----   Minimized:5 states - 0ms.
  Minimized:5 states - 0ms.
 quantified:5 states - 0ms
 fixing leading zeros:5 states
  determinizing:5 states
  determinized:5 states - 0ms
-----   Minimizing: 5 states.
   determinizing:5 states
   determinized:5 states - 0ms
-----   Minimized:5 states - 0ms.
  Minimized:5 states - 0ms.
 fixed leading zeros:5 states - 0ms
computed (a+b)+4
computing ((a+b)+4)-1
 computing &:2 states - 5 states
 computing cross product:2 states - 5 states
 computed cross product:10 states - 0ms
-----   Minimizing: 10 states.
   de

In [54]:
def aux "c=a+b+4-1":
eval comptot "Ec $aux(a,b,c) & 5>c"::

c=(((a+b)+4)-1):4 states - 1ms
Total computation time: 2ms.

computing aux(...)
 fixing leading zeros:4 states
  determinizing:4 states
  determinized:4 states - 0ms
-----   Minimizing: 4 states.
   determinizing:4 states
   determinized:4 states - 0ms
-----   Minimized:4 states - 0ms.
  Minimized:4 states - 0ms.
 fixed leading zeros:4 states - 0ms
computed aux(a,b,c))
computing 5>c
computed 5>c
5>c:4 states - 0ms
 computing aux(a,b,c))&5>c
  computing &:4 states - 4 states
  computing cross product:4 states - 4 states
  computed cross product:10 states - 0ms
-----    Minimizing: 10 states.
    determinizing:10 states
    determinized:10 states - 0ms
-----    Minimized:4 states - 0ms.
   Minimized:4 states - 0ms.
  computed &:4 states - 0ms
 computed aux(a,b,c))&5>c
 (aux(a,b,c))&5>c):4 states - 1ms
  computing quantifier E
   quantifying:4 states
-----     Minimizing: 4 states.
     determinizing:4 states
     determinized:4 states - 0ms
-----     Minimized:4 states - 0ms.
    Minimiz

In [43]:
%showme comp
%showme comptot

## Opérations arithmétiques :  faiblesse de l'associativité à gauche

Les <b class="cool">constantes arithmétiques</b> sont traités récursivement : 
* quel que soit le système de numération, l'automate pour le prédicat $(a):~ a=0$ est celui acceptant $0^*$.
* $(a):~ a = n$ avec $n \in \mathbb{N^*}$ est équivalent à $(a):~ Eb,c~ a=b+c ~\&~ b=n' ~\&~ c=n''$ avec $n'=\lfloor n \rfloor$ et $n''=\lceil n \rceil$.

Les opérations arithmétiques avec des constantes peuvent ainsi être traitées :
l'automate du prédicat $(a,b):~ a=b+n$ avec $n \in \mathbb{N}$ est l'automate du prédicat équivalent 
$(a,b):~ Ec~ a=b+c ~\&~ c=n$.

Similairement, la multiplication est traitée comme une succession d'additions :
le pédicat $(a,b):~ a=b*n$ avec $n \in \mathbb{N^*}$ est équivalent à 
$(a,b):~ Ec,d~ a=c+d ~\&~ c=b*n' ~\&~ d=b*n''$ avec $n'=\lfloor n \rfloor$ et $n''=\lceil n \rceil$.

⚠️ Pour Walnut, les opérations arithmétiques sont <b class="cool">associatives à droite</b>. 

Comme les nombre traités sont des entiers naturels, 

ce détail trouve son importance avec la soustraction.

<img style="display:block; position:absolute; float:right; right:0px; bottom:0px; z-index:2;" src="img/arithm/tree.svg">


#### Exemple $~(a):~ 0<=a-1+1~$ vs $~(a):~ 0<=a+1-1$

In [5]:
def right "0<=a+1-1";
def wrong "0<=a-1+1";
eval equiv "Aa $right(a) <=> $wrong(a)";



_____
FALSE



In [6]:
%showme right
%showme wrong

Décomposition de $~(a):~ 0<=a-1+1~$ comme il est traité par Walnut :

In [10]:
def minusFirst "b=a-1";
def plusSecond "Ea c=b+1 & $minusFirst(a,b)";
def wrongComplete "Ea 0<=b & $plusSecond(a,b)";

In [9]:
%showme minusFirst
%showme plusSecond
%showme wrongComplete

In [14]:
def plusFirst "b=a+1";
def minusSecond "Ea c=b-1 & $plusFirst(a,b)";
def rightComplete "Ea 0<=b & $minusSecond(a,b)";

In [15]:
%showme plusFirst
%showme minusSecond
%showme rightComplete

## Plan de route

* <span class="gray">Les opérations sur les prédicats,</span>
    * <span class="gray">Les opérateurs de logique d'arité 2</span>
    * <span class="gray">la négation</span>
    * <span class="gray">La quantification</span>
* <span class="gray">Les opérations sur les termes arithmétiques,</span>
* <b class="cool">L'appel à des prédicats préfinis,</b>
* <span class="">Les opérations sur les termes d'indexation de mots automatiques.</span>

## Appel d'un prédicat prédéfini

Comme indiqué précedemment, Walnut crée des automates intermédiares pour chaque sous formule dont il a besoin pour traiter un prédicat. Le mot clef <b class="cool"> def </b> permet de stoquer l'automate correspondant au prédicat traité et ainsi éviter d'avoir à reconstruire ces automates intermédiaires. Ces prédicats enregistrés peuvent être appelés au moyen d'un <b class="cool"> $ </b>.

L'appel de l'automate $M$ d'un prédicat $P(x_1, \cdots, x_n)$ s'écrit donc $ \$M(e_1, \cdots, e_n)$ avec $e_i$ un terme arithmétique ou un prédicat avec une seule variable libre et l'automate correspondant à cet appel est celui résultant des opérations :

$\text{E} x_1, \cdots, x_n~~ P ~\&~ (x_1=e_1') ~\&~ \cdots ~\&~ (x_n=e_n') ~\&~ e_{j_1} ~\&~ \cdots ~\&~ e_{j_k}$ avec

* $x_1, \cdots, x_n$ les variables libres de $P$,
* $j_1, \cdots, j_k$ les indices des $k$ prédicats parmi $e_1, \cdots, e_n$,
* pour $1 \leq i \leq n,~ e_i'=e_i$ si $e_i$ est un terme arithmétique et $e_i'=$ la variable libre de $e_i$ si $e_i$ est un prédicat.

In [63]:
def factorEq "Ak k<n => T[i+k]=T[j+k]";
eval factor "$factorEq(i,j,n)"::


computing factorEq(...)
 fixing leading zeros:14 states
  determinizing:14 states
  determinized:14 states - 0ms
-----   Minimizing: 14 states.
   determinizing:14 states
   determinized:14 states - 0ms
-----   Minimized:14 states - 0ms.
  Minimized:14 states - 0ms.
 fixed leading zeros:14 states - 0ms
computed factorEq(i,j,n))
Total computation time: 1ms.



## Plan de route

* <span class="gray">Les opérations sur les prédicats,</span>
    * <span class="gray">Les opérateurs de logique d'arité 2</span>
    * <span class="gray">la négation</span>
    * <span class="gray">La quantification</span>
* <span class="gray">Les opérations sur les termes arithmétiques,</span>
* <span class="gray">L'appel à des prédicats préfinis,</span>
* <b class="cool">Les opérations sur les termes d'indexation de mots automatiques.</b>

## Indexation des suites automatiques

Pour Walnut, une suite automatique est un <b class="cool">automate fini déterminite à sortie</b>.
Ainsi un suite $W$ est représentée par un DFAO $M = (Q, \S{1}, \cdots, \S{n},\Gamma, q_0, \delta, \pi)$ avec 

* $Q$ son ensemble fini des états,
* $\S{i}$ l'alphabet de $S_i$ avec $1 \leq i \leq n$ et où $S_i$ est le système de numération la $i$-ème dimension,
* $\Gamma$ son alphabet fini de sortie,
* $q_0 \in Q$ son état initial,
* $\delta : Q \times \S{1} \times \cdots \times \S{n} \rightarrow Q$ sa fonction de transition,
* $\pi : Q \rightarrow \Gamma$ sa fonction de sortie.

Dans Walnut les prédicats sont des DFA, un prédicat peut être obtenu 

L'automate représentant la formule $(x_1, \cdots, x_n):~ W[x_1]\cdots[x_n] = @a$ avec $a \in \Gamma$ est le DFA correspondant à la minimisation de $A = (Q, \S{1}, \cdots, \S{n}, q_0, F, \delta)$ avec 

* $F = \lbrace q ~|~ \pi(q)=a \rbrace$

On peut désormais utiliser tous les outils définis précédement pour traiter les suites automatiques.

Le produit synchronisé notamment est étendu sur les DFAO avec $\pi''$ la fonction de production du produit de deux DFAO (de fct de prod. $\pi$ et $\pi'$) :

$\pi'' : Q \times Q' \rightarrow \Gamma''$

<span class="trans"> $\pi'':$</span> $(q,q') \mapsto \pi(q) \diamond \pi'(q')$

In [12]:
eval test "T[0] * T[1]=@0":

T[0]=@0:2 states - 1ms
Total computation time: 1ms.
____
TRUE



In [1]:
def factorEq "Ak k<n => T[i+k]=T[j+k]";
def factorBis "Ak k<n => (Ex T[i+k]=x & T[j+k]=x)":
eval equiv "Ai,j,n $factorEq(i,j,n) <=> $factorBis(i,j,n)":


k<n:2 states - 0ms
computed ~:1 states - 13ms
computed ~:1 states - 1ms
computed ~:2 states - 2ms
 T[(i+k)]:5 states - 24ms
  T[(j+k)]:5 states - 3ms
   (T[(i+k)]&T[(j+k)]):17 states - 3ms
    (E x (T[(i+k)]&T[(j+k)])):8 states - 3ms
     (k<n=>(E x (T[(i+k)]&T[(j+k)]))):17 states - 4ms
      (A k (k<n=>(E x (T[(i+k)]&T[(j+k)])))):14 states - 29ms
Total computation time: 66ms.

(factorEq(i,j,n))<=>factorBis(i,j,n))):1 states - 2ms
 (A i , j , n (factorEq(i,j,n))<=>factorBis(i,j,n)))):1 states - 1ms
Total computation time: 6ms.
____
TRUE



## Conclusion

Pour résumer, voici une décomposition de la construction de l'automate d'un prédicat par Walnut :

In [33]:
def factorEq "Ak k<n => T[i+k]=T[j+k]";

In [37]:
def plus "z=i+k";
def thue "Ex z=T[x] & $plus(i,k,x)";
def eq "$thue(i,k,z) & $thue(j,k,z)";
def inf "k<n";
def implique "Ez $inf(k,n) => $eq(i,j,k,z)";
def forall "Ak $implique(i,j,k,n)";

In [41]:
eval same "Ai,j,n $factorEq(i,j,n) <=> $forall(i,j,n)";

____
TRUE

